In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.exceptions import ConvergenceWarning
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
# from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor





XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=0, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [112]:
# Load the data
df = pd.read_csv('data/get_around_pricing_project.csv', index_col=0)

print(df.head())

  model_key  mileage  engine_power    fuel paint_color     car_type   
0   Citroën   140411           100  diesel       black  convertible  \
1   Citroën    13929           317  petrol        grey  convertible   
2   Citroën   183297           120  diesel       white  convertible   
3   Citroën   128035           135  diesel         red  convertible   
4   Citroën    97097           160  diesel      silver  convertible   

   private_parking_available  has_gps  has_air_conditioning  automatic_car   
0                       True     True                 False          False  \
1                       True     True                 False          False   
2                      False    False                 False          False   
3                       True     True                 False          False   
4                       True     True                 False          False   

   has_getaround_connect  has_speed_regulator  winter_tires   
0                   True                 

In [113]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 4843 entries, 0 to 4842
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   model_key                  4843 non-null   object
 1   mileage                    4843 non-null   int64 
 2   engine_power               4843 non-null   int64 
 3   fuel                       4843 non-null   object
 4   paint_color                4843 non-null   object
 5   car_type                   4843 non-null   object
 6   private_parking_available  4843 non-null   bool  
 7   has_gps                    4843 non-null   bool  
 8   has_air_conditioning       4843 non-null   bool  
 9   automatic_car              4843 non-null   bool  
 10  has_getaround_connect      4843 non-null   bool  
 11  has_speed_regulator        4843 non-null   bool  
 12  winter_tires               4843 non-null   bool  
 13  rental_price_per_day       4843 non-null   int64 
dtypes: bool(7), i

In [114]:
print("Percentage of missing values: ")
display(100*df.isnull().sum()/df.shape[0])

Percentage of missing values: 


model_key                    0.0
mileage                      0.0
engine_power                 0.0
fuel                         0.0
paint_color                  0.0
car_type                     0.0
private_parking_available    0.0
has_gps                      0.0
has_air_conditioning         0.0
automatic_car                0.0
has_getaround_connect        0.0
has_speed_regulator          0.0
winter_tires                 0.0
rental_price_per_day         0.0
dtype: float64

In [115]:
# Extract the features
X = df.drop('rental_price_per_day', axis=1)

# Extract the target column
y = df.loc[:, 'rental_price_per_day']

# Train / test split 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size = 0.2)

In [116]:
# determine categorical and numerical features
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object', 'bool']).columns

# Numerical Transformer
numerical_transformer = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
])

# Categorical Transformer
categorical_transformer = Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder(drop='first', handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("numerical_transformer", numerical_transformer, numerical_features),
        ("categorical_transformer", categorical_transformer, categorical_features)
    ]
)

# X_train = preprocessor.fit_transform(X_train)
# X_test = preprocessor.transform(X_test)

In [117]:
import warnings
import re
from contextlib import contextmanager

@contextmanager
def ignore_unknown_categories_warning():
    default_showwarning = warnings.showwarning

    def custom_warning_message(message, category, *args, **kwargs):
        if not (category == UserWarning and re.match(r"^Found unknown categories.*", str(message))):
            default_showwarning(message, category, *args, **kwargs)

    warnings.showwarning = custom_warning_message
    try:
        yield
    finally:
        warnings.showwarning = default_showwarning

In [118]:
# List of models
models = [
    LinearRegression(),
    Ridge(),
    Lasso(),
    ElasticNet(),
    SVR(),
    DecisionTreeRegressor(),
    RandomForestRegressor(),
    GradientBoostingRegressor(),
    XGBRegressor(),
    # LGBMRegressor(),
    CatBoostRegressor(verbose=0)
]

# List of param_grids for each model
param_grids = [
    {},  # Linear Regression
    {'model__alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}, # Ridge
    {'model__alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}, # Lasso
    {'model__alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000], 'model__l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9]}, # ElasticNet
    {'model__kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'model__C': [0.1, 1, 10, 100], 'model__epsilon': [0.01, 0.1, 1, 10]}, # SVR
    {'model__max_depth': [2, 5, 10, 20], 'model__min_samples_split': [2, 5, 10]}, # Decision Tree Regressor
    {'model__n_estimators': [100, 200, 500], 'model__max_depth': [2, 5, 10, 20], 'model__min_samples_split': [2, 5, 10]}, # Random Forest Regressor
    {'model__n_estimators': [100, 200, 500], 'model__learning_rate': [0.01, 0.1, 0.2], 'model__max_depth': [2, 5, 10], 'model__min_samples_split': [2, 5, 10]}, # Gradient Boosting Regressor
    {'model__n_estimators': [100, 200, 500], 'model__learning_rate': [0.01, 0.1, 0.2], 'model__max_depth': [2, 5, 10], 'model__min_child_weight': [1, 5, 10],'model__gamma': [0, 0.5, 1]}, # XGB Regressor
    # {'n_estimators': [100, 200, 500], 'learning_rate': [0.01, 0.1, 0.2], 'max_depth': [2, 5, 10], 'num_leaves': [31]}, # LGBM Regressor
    {'model__iterations': [100, 200, 500], 'model__learning_rate': [0.01, 0.1, 0.2], 'model__depth': [3, 6, 10], 'model__l2_leaf_reg': [1, 3, 5]}, # CatBoost Regressor
]

# Initialize an empty DataFrame to store the results
results_df = pd.DataFrame(columns=['Model', 'Best_Params', 'Best_Score'])

results = []

for i, model in enumerate(models):
    with ignore_unknown_categories_warning():
        param_grid = param_grids[i]
        
        # Create a pipeline with the preprocessor and the model
        pipeline = Pipeline([
            ('preprocessor', preprocessor),
            ('model', model)
        ])

        # Perform grid search with the current model and its param_grid
        grid = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1)
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            grid.fit(X_train, y_train)

        # Store the best estimator and its score
        best_estimator = grid.best_estimator_
        best_score = grid.best_score_
        
        results.append((best_estimator, best_score))

        # Make predictions on train and test sets
        y_train_pred = best_estimator.predict(X_train)
        y_test_pred = best_estimator.predict(X_test)

        # Calculate RMSE on train and test sets
        train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
        test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

        # Calculate R2 score on train and test sets
        train_r2 = r2_score(y_train, y_train_pred)
        test_r2 = r2_score(y_test, y_test_pred)

        # Add the R2 scores to the DataFrame
        results_df = pd.concat([results_df, pd.DataFrame({
            'Model': [model.__class__.__name__],
            'Best_Params': [grid.best_params_],
            'Best_Score': [best_score],
            'Train_RMSE': [train_rmse],
            'Test_RMSE': [test_rmse],
            'Train_R2': [train_r2],
            'Test_R2': [test_r2]
        })], ignore_index=True)
        
        print(f"Best parameters for {model.__class__.__name__}: {grid.best_params_}")
    
# Print the results DataFrame
print(results_df)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


/Users/pryda/.pyenv/versions/3.10.9/envs/bloc_5/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/pryda/.pyenv/versions/3.10.9/envs/bloc_5/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/pryda/.pyenv/versions/3.10.9/envs/bloc_5/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/pryda/.pyenv/versions/3.10.9/envs/bloc_5/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be en

Best parameters for LinearRegression: {}
Fitting 5 folds for each of 7 candidates, totalling 35 fits


/Users/pryda/.pyenv/versions/3.10.9/envs/bloc_5/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/pryda/.pyenv/versions/3.10.9/envs/bloc_5/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/pryda/.pyenv/versions/3.10.9/envs/bloc_5/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/pryda/.pyenv/versions/3.10.9/envs/bloc_5/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be en

Best parameters for Ridge: {'model__alpha': 1}
Fitting 5 folds for each of 7 candidates, totalling 35 fits


...

Best parameters for GradientBoostingRegressor: {'model__learning_rate': 0.1, 'model__max_depth': 5, 'model__min_samples_split': 10, 'model__n_estimators': 200}
Fitting 5 folds for each of 243 candidates, totalling 1215 fits


/Users/pryda/.pyenv/versions/3.10.9/envs/bloc_5/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/pryda/.pyenv/versions/3.10.9/envs/bloc_5/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/pryda/.pyenv/versions/3.10.9/envs/bloc_5/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/pryda/.pyenv/versions/3.10.9/envs/bloc_5/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be en

Best parameters for XGBRegressor: {'model__gamma': 0, 'model__learning_rate': 0.1, 'model__max_depth': 10, 'model__min_child_weight': 5, 'model__n_estimators': 100}
Fitting 5 folds for each of 81 candidates, totalling 405 fits


/Users/pryda/.pyenv/versions/3.10.9/envs/bloc_5/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/pryda/.pyenv/versions/3.10.9/envs/bloc_5/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/pryda/.pyenv/versions/3.10.9/envs/bloc_5/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/pryda/.pyenv/versions/3.10.9/envs/bloc_5/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be en

Best parameters for CatBoostRegressor: {'model__depth': 6, 'model__iterations': 500, 'model__l2_leaf_reg': 3, 'model__learning_rate': 0.1}
                       Model   
0           LinearRegression  \
1                      Ridge   
2                      Lasso   
3                 ElasticNet   
4                        SVR   
5      DecisionTreeRegressor   
6      RandomForestRegressor   
7  GradientBoostingRegressor   
8               XGBRegressor   
9          CatBoostRegressor   

                                         Best_Params  Best_Score  Train_RMSE   
0                                                 {} -343.907239   18.094369  \
1                                {'model__alpha': 1} -342.667244   18.115548   
2                            {'model__alpha': 0.001} -343.674886   18.096215   
3    {'model__alpha': 0.001, 'model__l1_ratio': 0.7} -342.722030   18.122744   
4  {'model__C': 100, 'model__epsilon': 10, 'model... -299.750061   13.928242   
5  {'model__max_depth': 10, 

In [2]:
pd.set_option('display.max_columns', None)
results_df

NameError: name 'pd' is not defined

In [1]:
print(results_df["Best_Params"][9])

NameError: name 'results_df' is not defined

### In the context of the grid search, best_score represents the highest mean score, based on the selected scoring metric (in this case, neg_mean_squared_error). The negative value is used because GridSearchCV minimizes the metric, so by taking the negative value we can still track the performance in a maximization setting. The lower the mean squared error, the better the model's performance.